## Task 5 - Inductive Biases of Models: Locality Biases

In [ ]:
!pip install timm

In [11]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms.v2 as transforms
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import timm
from typing import Optional

In [8]:
TRAIN_TFMS = transforms.Compose([
    transforms.RandAugment(),
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
TEST_TFMS = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

### Evaluating for Scrambled Image

In [ ]:
!python3 drive/MyDrive/ATML/PA1/task5/scrambled_images.py --model_name 'vit_small_16' --out_dir 'drive/MyDrive/ATML/PA1/task5' --dataset 'CIFAR-10' --patch_size 56 --epochs 2

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`. Output is equivalent up to float precision.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`. Output is equivalent up to float precision.
  warnings.warn(
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Training: [1563/1563] Loss: 1.2724 Acc: 0.7342
Evaluation: [313/313] Loss: 0.2022 Acc: 0.9384
============ Epoch 1 --> Train Acc: 0.7342 || Test Acc: 0.9384 || Time: 226.71 s ============

Training: [1563/1563] Loss: 0.2878 Ac

### Evaluating for Noise Injection

In [9]:
# Custom transform which injects local noise
class AddNoiseToPatch:
    def __init__(self, noise_level=0.1, patch_coords=(0, 0, 50, 50)):
        self.noise_level = noise_level
        self.patch_coords = patch_coords  # (x1, y1, x2, y2)

    def __call__(self, img):
        # Convert to numpy array
        img_np = np.array(img)

        # Extract patch coordinates
        x1, y1, x2, y2 = self.patch_coords

        # Generate random noise
        noise = np.random.normal(0, self.noise_level, img_np[y1:y2, x1:x2].shape).astype(np.uint8)

        # Add noise to the patch
        img_np[y1:y2, x1:x2] = np.clip(img_np[y1:y2, x1:x2] + noise, 0, 255)

        # Convert back to PIL Image
        return Image.fromarray(img_np)

def get_noised_data(noise_size, root):

    NOISE_TEST_TFMS = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),
        AddNoiseToPatch(noise_level=25, patch_coords=(50, 50, 50+noise_size, 50+noise_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

    trainset = torchvision.datasets.CIFAR10(
        root, train=True, download=True, transform=TRAIN_TFMS
    )

    normal_testset = torchvision.datasets.CIFAR10(
        root, train=False, download=True, transform=TEST_TFMS
    )

    noised_testset = torchvision.datasets.CIFAR10(
        root, train=False, download=True, transform=NOISE_TEST_TFMS
    )

    return trainset, normal_testset, noised_testset

In [10]:
@torch.inference_mode()
def eval_step(model, dataloader, criterion, device):
    '''Evaluate the model'''

    model.eval()

    eval_loss = 0.0
    eval_acc = 0.0

    for i, data in enumerate(dataloader):

        X, y = data[0].to(device), data[1].to(device)

        logits = model(X)
        loss = criterion(logits, y)
        eval_loss += loss.item()

        y_pred = torch.argmax(logits.detach(), dim=1)
        eval_acc += (y_pred == y).sum().item() / len(y)

        # Print dynamic progress on the same line using \r
        print(f'\rEvaluation: [{i+1}/{len(dataloader)}] '
              f'Loss: {eval_loss / (i + 1):.4f} '
              f'Acc: {eval_acc / (i + 1):.4f}', end='')

    eval_loss = eval_loss / len(dataloader)
    eval_acc = eval_acc / len(dataloader)

    # Move to the next line after the loop is done
    print()

    return eval_loss, eval_acc

In [15]:
saved_path = 'drive/MyDrive/ATML/PA1/model/cifar10_final.pth'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = timm.create_model('vit_small_patch16_224', pretrained=False)
model.load_state_dict(torch.load(saved_path, weights_only=True))
model.to(device)

criterion = torch.nn.CrossEntropyLoss()

In [19]:
def get_dataloader(dataset: Dataset,
                   batch_size: int,
                   is_train: bool,
                   num_workers: int = 1):

    loader = DataLoader(dataset, batch_size=batch_size,
                        shuffle=is_train, num_workers=num_workers)
    return loader

_, normal_testset, noised_testset = get_noised_data(noise_size=100, root='drive/MyDrive/ATML/PA1')
normal_dl = get_dataloader(normal_testset, batch_size=64, is_train=False)
noise_dl = get_dataloader(noised_testset, batch_size=64, is_train=False)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [21]:
normal_loss, normal_acc = eval_step(model, normal_dl, criterion, device)
noise_loss, noise_acc = eval_step(model, noise_dl, criterion, device)

print(f'\n====================== Normal Accuracy: {normal_acc*100:.3f}% || Noise Injected Accuracy: {noise_acc*100:.3f}% ======================')

Evaluation: [157/157] Loss: 0.1270 Acc: 0.9592
Evaluation: [157/157] Loss: 0.2346 Acc: 0.9293

====================== Normal Accuracy: 95.920% || Noise Injected Accuracy: 92.934% ======================


### Evaluating Global Style Changes

In [ ]:
def get_custom_data(path: str):
    dataset = datasets.ImageFolder(path, transform=TEST_TFMS)

    return dataset


global_style_testset = get_custom_data("../input/cifar10/stylized_cifar10")
global_style_dl = get_dataloader(
    global_style_testset, batch_size=64, is_train=False, num_workers=2
)

In [ ]:
normal_loss, normal_acc = eval_step(model, normal_dl, criterion, device)
global_style_loss, global_style_acc = eval_step(model, global_style_dl, criterion, device)

print(f"====================== Normal Accuracy: {normal_acc*100:.4f}% || Global Style Accuracy: {global_style_acc*100:.4f}% ======================")

Evaluation: [157/157] Loss: 0.1533 Acc: 0.9493
Evaluation: [8/8] Loss: 1.3926 Acc: 0.5575
====================== Normal Accuracy: 94.9343% || Global Style Accuracy: 55.7542% ======================
